## Optimización de Selección de Proyectos

#### Descripción del problema:
Se necesita decidir qué proyectos financiar, cuando existe una limitación estricta en la *disponibilidad de capital*.

Para ello se define como objetivo maximizar el valor presente neto total (NPV) de la selección de proyectos. Este indicador puede modificarse a uno más adecuado al contexto.


### Formulación del Problema de Optimización

##### Conjuntos y Notaciones:
- Sea $n$ el número de proyectos a evaluar
- Sea $N = \{1,...,n\}$ el conjunto de proyectos.

#### Parámetros
- Sea $NPV_i$ el Valor Presente Neto (NPV) del proyecto $i$.
- Sea $COST_i$ el costo requerido para financiar el proyecto $i$.
- Sea $BUDGET$ el presupuesto total disponible.

#### Variables de Decisión
- Sea $x_i \in \{0,1\}$, 1 si el proyecto $i$ es seleccionado y 0 en otro caso.

#### Función Objetivo
La función objetivo se define como la suma de los valores presentes netos de los proyectos seleccionados. Buscamos maximizar esta cantidad sujeta a la restricción de presupuesto.

$\begin{align}
\max\limits_{\forall i \in N} \sum_{n=1}^{N} NPV_i x_i
\end{align}$

Sujeta a:

$\begin{align}
\sum_{n=1}^{N} COST_i x_i \leq BUDGET \\
x_i \in \{0,1\}, \forall i \in N
\end{align}$

In [1]:
# librerías de optimización

import gurobipy as grb
import numpy as np
import pandas as pd

### Ejemplo
Suponga que tiene el siguiente portafolio de proyectos, con sus respectivos costos y NPV's.

***"Sí solamente tengo 260M de presupuesto, qué proyectos debería seleccionar para maximizar el NPV total?"***

In [2]:
df = pd.read_csv(r'capital.csv')
df

,ID,NPV (MM USD),COST (MM USD)
0,1,100,80
1,2,50,45
2,3,175,150
3,4,100,80
4,5,110,100


#### Parámetros del Problema de Optimización

In [3]:
# proyectos
Proyectos = np.array(df)

# número de proyectos
n = len(df['ID'])

# lista de proyectos
N = {i for i in range(1,n+1)}

# parámetros
NPV = {i:Proyectos[i-1,1] for i in N}
COST = {i:Proyectos[i-1,2] for i in N}
BUDGET = 260

#### Definición del Modelo de Optimización

In [4]:
# modelo de optimización
model = grb.Model("Seleccion de Inversiones")

# variables de decisión
x = model.addVars(N, vtype = grb.GRB.BINARY)


Set parameter Username


#### Función Objetivo

In [5]:
# función objetivo
model.setObjective(grb.quicksum(NPV[i]*x[i] for i in N), grb.GRB.MAXIMIZE)

#### Restricciones
Nota: La restricción asociada al tipo binario de la variable de selección $x_i$ se incluyó en la declaración de la variable como tipo GRB.BINARY

In [6]:
# restricciones
model.addConstr(grb.quicksum(COST[i]*x[i]for i in N)<=BUDGET)
model.update()

### Solución al Problema de Optimización de Programación Entera-Mixta (MIP)

In [7]:
# optimización
model.optimize()

if model.status != grb.GRB.status.OPTIMAL:
    print("No se encontró una solución factible...")
else:
    print("El valor optimo de la función objetivo es %g"%model.objVal)
    resultado_objetivo = model.objVal
    resultado_seleccion_proyectos = [i for i in N if x[i].X >= 1]

    df_salida = pd.DataFrame({"valor función objetivo": [resultado_objetivo],
                              "proyectos seleccionados": [resultado_seleccion_proyectos]})

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x51d44574
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [4e+01, 2e+02]
  Objective range  [5e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 3e+02]
Found heuristic solution: objective 250.0000000
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 310 250 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.100000000000e+02, best bound 3.100000000000e+02, gap 0.0000%
El valor optimo de la función objetivo es 310


In [8]:
df_salida

,valor función objetivo,proyectos seleccionados
0,310.0,"[1, 4, 5]"


Por lo tanto, dados los paarametros de NPV y de COSTO para cada proyecto, si se cuenta con un presupuesto de 260MM, se deberían seleccionar los proyectos *1, 4 y 5*. Esta selección maximiza el NPV total a un valor de 310 MM.